# 数据预处理

- 分度数处理，对95%和5%的极端的两头进行抹平操作

In [1]:
import pandas as pd

In [2]:
data_FN = 'C:/Users/pangbochen/Documents/finance/Efund/DT_RF/data.csv'

In [3]:
data_df = pd.read_csv(data_FN, index_col=0, dtype={'stock_id':str})

In [4]:
data_df.head()

,date,stock_id,ROE,RealizedVolatility_3M,EBIT2EV,ILLIQ,InvTurnover,OCFGrowth_YOY,AmountVol_1M_12M,ProfitGrowth_Qr_YOY,...,NetMargin,EP_FY1,CGO_3M,CFP_TTM,MaxRet,MR,EquityGrowth_YOY,EP2_TTM,ROIC,return
0,20060131,000001,0.0403,0.0134,NaN,19.237,NaN,0.0407,1.2669,0.3101,...,0.0450,NaN,0.0456,0.5838,0.0147,NaN,0.1470,0.0139,NaN,NaN
1,20060131,000002,0.2095,0.0248,0.0576,12.110,0.2585,0.6083,1.6922,0.0908,...,0.1569,NaN,0.1290,0.0971,0.0551,NaN,0.1802,0.0749,0.1437,NaN
2,20060131,000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20060131,000004,-0.0769,0.0256,-0.0091,555.110,0.8123,0.1103,1.7020,-3.2726,...,-0.1338,NaN,-0.0782,0.0022,0.1002,NaN,-0.0777,-0.0190,-0.0436,NaN
4,20060131,000005,-0.0284,0.0232,-0.0155,180.470,0.0969,-0.5478,0.9684,NaN,...,-0.8403,NaN,0.0263,0.0234,0.0117,NaN,-0.0770,-0.0546,-0.0454,NaN


In [5]:
feature_list = data_df.columns

In [6]:
feature_list  = feature_list[2:]

# 进行因子两头的抹平操作

In [7]:
tmp_series = data_df['ROE'].copy()

In [8]:
tmp_series.describe()

count    334680.000000
mean          0.071361
std           0.444195
min          -9.864200
25%           0.028700
50%           0.075500
75%           0.129700
max           9.647900
Name: ROE, dtype: float64

In [9]:
def func(x, high=1, low=1):
    if x > high:
        return high
    elif x < low:
        return low
    else:
        return x

In [10]:
high = tmp_series.quantile(0.95)
high

0.2621

In [11]:
low = tmp_series.quantile(0.05)
low

-0.1327

In [12]:
tmp_series.apply(lambda x: func(x, high, low)).describe()

count    334680.000000
mean          0.078022
std           0.090952
min          -0.132700
25%           0.028700
50%           0.075500
75%           0.129700
max           0.262100
Name: ROE, dtype: float64

## NA

In [13]:
data_df = data_df.dropna()

## 分度极端

In [14]:
high_quantile = 0.95
low_quantile = 0.05

In [15]:
for feature in feature_list:
    high = data_df[feature].quantile(high_quantile)
    low  = data_df[feature].quantile(low_quantile)
    data_df[feature] = data_df[feature].apply(lambda x: func(x, high, low))

In [16]:
data_df.head()

,date,stock_id,ROE,RealizedVolatility_3M,EBIT2EV,ILLIQ,InvTurnover,OCFGrowth_YOY,AmountVol_1M_12M,ProfitGrowth_Qr_YOY,...,NetMargin,EP_FY1,CGO_3M,CFP_TTM,MaxRet,MR,EquityGrowth_YOY,EP2_TTM,ROIC,return
148462,20090630,000002,0.1310,0.0279,0.0483,3.42212,0.17716,3.5949,1.4888,0.07050,...,0.0955,3.6145,0.1194,0.0362,0.0969,15.432,0.0823,0.0293,0.0973,0.232688
148466,20090630,000006,0.0255,0.0332,0.0095,19.68400,0.17716,0.5792,1.2839,-0.73090,...,0.0136,4.3216,0.1194,-0.0575,0.0821,14.416,0.0010,0.0029,0.0083,0.167348
148472,20090630,000012,0.0627,0.0327,0.0230,4.38850,1.95110,1.2827,2.0421,-0.68680,...,0.0683,4.0335,-0.0844,0.0608,0.0272,14.667,0.1000,0.0130,0.0184,-0.131858
148481,20090630,000021,0.0842,0.0340,0.0263,12.86900,6.38140,1.0858,2.2975,-0.42840,...,0.0239,3.1594,0.1194,0.0364,0.0571,14.613,0.0010,0.0268,0.0916,0.035819
148484,20090630,000024,0.1180,0.0327,0.0244,6.58880,0.17716,1.0687,2.1200,2.80814,...,0.3178,3.1311,0.1194,-0.0449,0.0672,15.231,0.8986,0.0253,0.0498,0.146248


In [28]:
output_FN = 'data_processed.csv'

In [26]:
data_df.to_csv(output_FN, index=False)

# load for test

In [27]:
new_df = pd.read_csv(output_FN, dtype={'stock_id':str})
new_df.head()

,date,stock_id,ROE,RealizedVolatility_3M,EBIT2EV,ILLIQ,InvTurnover,OCFGrowth_YOY,AmountVol_1M_12M,ProfitGrowth_Qr_YOY,...,NetMargin,EP_FY1,CGO_3M,CFP_TTM,MaxRet,MR,EquityGrowth_YOY,EP2_TTM,ROIC,return
0,20090630,000002,0.1310,0.0279,0.0483,3.42212,0.17716,3.5949,1.4888,0.07050,...,0.0955,3.6145,0.1194,0.0362,0.0969,15.432,0.0823,0.0293,0.0973,0.232688
1,20090630,000006,0.0255,0.0332,0.0095,19.68400,0.17716,0.5792,1.2839,-0.73090,...,0.0136,4.3216,0.1194,-0.0575,0.0821,14.416,0.0010,0.0029,0.0083,0.167348
2,20090630,000012,0.0627,0.0327,0.0230,4.38850,1.95110,1.2827,2.0421,-0.68680,...,0.0683,4.0335,-0.0844,0.0608,0.0272,14.667,0.1000,0.0130,0.0184,-0.131858
3,20090630,000021,0.0842,0.0340,0.0263,12.86900,6.38140,1.0858,2.2975,-0.42840,...,0.0239,3.1594,0.1194,0.0364,0.0571,14.613,0.0010,0.0268,0.0916,0.035819
4,20090630,000024,0.1180,0.0327,0.0244,6.58880,0.17716,1.0687,2.1200,2.80814,...,0.3178,3.1311,0.1194,-0.0449,0.0672,15.231,0.8986,0.0253,0.0498,0.146248
